In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import h5py
import shutil
import cv2

2024-07-10 19:36:01.353298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 19:36:01.353453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 19:36:01.567561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
DIR = "/kaggle/input/isic-2024-challenge"
print(os.listdir(DIR))

['sample_submission.csv', 'train-metadata.csv', 'test-metadata.csv', 'test-image.hdf5', 'train-image', 'train-image.hdf5']


In [3]:
train_dir = os.path.join(DIR,"train-image","image")

In [4]:
# Number of images
len(os.listdir(train_dir))

401059

In [5]:
training_meta_data_path = os.path.join(DIR,"train-metadata.csv")

In [6]:
training_meta_data = pd.read_csv(training_meta_data_path)

/tmp/ipykernel_18/367696351.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  training_meta_data = pd.read_csv(training_meta_data_path)


In [7]:
training_meta_data

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,0,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,IL_9520694,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999988
401055,ISIC_9999951,0,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999820
401056,ISIC_9999960,0,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,...,IL_9852274,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999416
401057,ISIC_9999964,0,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000


In [8]:
test_meta_data_path = os.path.join(DIR,"test-metadata.csv")

In [9]:
test_meta_data = pd.read_csv(test_meta_data_path)

In [10]:
test_meta_data

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.304827,1.281532,31.114600,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.000000,1.271940,68.141071,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY
2,ISIC_0015740,IP_7142616,65.0,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,19.937380,...,0.230742,1.080308,41.358640,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY


In [11]:
# installing package h5py https://docs.h5py.org/en/stable/quick.html
!pip install h5py

In [12]:
training_file = h5py.File(os.path.join(DIR,"train-image.hdf5"))

In [13]:
training_meta_data_benign = training_meta_data[training_meta_data["target"]==0]

In [14]:
#Count of Benign Instances
len(training_meta_data_benign)

400666

In [15]:
training_meta_data_malignant = training_meta_data[training_meta_data["target"]==1]

In [16]:
len(training_meta_data_malignant)

393

In [17]:
new_training_data="new_training_data"
malignant_dir = "malignant"
benign_dir = "benign"
os.mkdir(os.path.join(new_training_data))
os.mkdir(os.path.join(new_training_data,malignant_dir))
os.mkdir(os.path.join(new_training_data,benign_dir))
new_training_data_malignant = training_meta_data_malignant['isic_id'].apply(lambda x: shutil.copy(os.path.join(train_dir,x+".jpg"),os.path.join(new_training_data,malignant_dir,x+".jpg")))
new_training_data_benign = training_meta_data_benign['isic_id'].apply(lambda x: shutil.copy(os.path.join(train_dir,x+".jpg"),os.path.join(new_training_data,benign_dir,x+".jpg")))